In [ ]:
#| hide
from sherlock.core import *

# Sherlock Domains Python SDK

> AI agent & SDK to buy and managed domains from [Sherlock Domains](https://www.sherlockdomains.com/)

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/sherlock-python.git
```

```sh
$ pip install sherlock-domains
```


[repo]: https://github.com/Fewsats/sherlock-python
[pypi]: https://pypi.org/project/sherlock-domains/


## How to use

Create a Sherlock instance with a private key for the agent to use. If no key is provided, a new one will be generated and saved to the default config file.

In [ ]:
s = Sherlock()
s

Sherlock(pubkey=90ba884688884277e49080712f386eebc88806efa8345ca937f75fe80950156d)

You can search for a domain and request to purchase it. Purchasing a domain requires contact information as mandated by ICANN.

In [ ]:
sr = s.search("trakwiska.com")
# Don't print the whole lists
sr['available'] = sr['available'][:1]
sr['unavailable'] = sr['unavailable'][:1]
sr

{'id': 'c004a96f-7c8f-4377-804b-81927c579746',
 'created_at': '2025-01-12T12:42:13.987Z',
 'available': [{'name': 'trakwiska.com',
   'tld': 'com',
   'tags': [],
   'price': 1105,
   'currency': 'USD',
   'available': True}],
 'unavailable': []}

Contact information is needed for registering a new domain with the ICANN.

In [ ]:
c = Contact(**{
    "first_name": "Test",
    "last_name": "User",
    "email": "test@example.com",
    "address": "123 Test St",
    "city": "Test City",
    "state": "CA",
    "country": "US",
    "postal_code": "12345",
})

s.set_contact_information(
    cfn=c.first_name,
    cln=c.last_name,
    cem=c.email,
    cadd=c.address,
    cct=c.city,
    cst=c.state,
    cpc=c.postal_code,
    ccn=c.country
)


{'message': 'Contact information updated successfully'}

Finalizing a purchase involves a payment. Sherlock Domains currently supports two payment methods: Credit Card (`credit_card`) and the Lightning Netowrk (`lightning`).
By default the credit card method is used. In order to complete the payment: 
-  `credit_card` returns a checkout URL.
-  `lightning` returns a lightning invoice.

In [ ]:
s.purchase_domain(sr['id'], "trakwiska.com")

{'payment_method': {'checkout_url': 'https://checkout.stripe.com/c/pay/cs_live_a1Hojo4GRRQJekZuVyKnO7kFizhuCAbt9kwE2tYlmL7K31mXHXHAGWHBlf#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl',
  'lightning_invoice': None},
 'expires_at': '2025-01-12T13:12:15.643Z'}

You can now use the checkout URL to complete the purchase and the domain will be registered to your agent.

You can view the domains you own with:

In [ ]:
s.domains()

[{'id': 'd9b2cc30-c15d-44b9-9d39-5d33da504484',
  'domain_name': 'h402.org',
  'created_at': '2024-12-28T18:58:49.899Z',
  'expires_at': '2024-12-31T18:58:42Z',
  'auto_renew': False,
  'locked': True,
  'private': True,
  'nameservers': [],
  'status': 'active'}]

Below is a list of all the tools that the client offers to manage the domains and purchases.

In [ ]:
s.as_tools().map(lambda t: t.__name__)

(#10) ['_me','_set_contact_information','_get_contact_information','_search','_purchase_domain','_domains','_dns_records','_create_dns_record','_update_dns_record','_delete_dns_record']

## AI agents

We will show how to enable your AI assistant to handle payments using [Claudette](https://claudette.answer.ai), Answer.ai convenient wrapper for Claude. You'll need to export your `ANTHROPIC_API_KEY`.

In [ ]:
from claudette import Chat, models

In [ ]:
model = models[1]; model

'claude-3-5-sonnet-20240620'

Create a Sherlock instance with a public & private key for the agent to use.

Sherlock supports returning all the tools with `s.as_tools()`.

In [ ]:
sp = 'You are a helpful assistant that has access to a domain purchase API.'
chat = Chat(model, sp=sp, tools=s.as_tools())

pr = f"Search if domain 'the-favourite-game.com' is available? If it is request a purchase and process the payment using credit card method."
r = chat.toolloop(pr, trace_func=print)
r

Message(id='msg_01M9UEkTSLZaWqKHgDpc4n8p', content=[TextBlock(text="Certainly! I'll search for the domain 'the-favourite-game.com' and if it's available, I'll proceed with the purchase request using the credit card payment method. Let's start with the search.", type='text'), ToolUseBlock(id='toolu_01KbnEirDQbNeEp7dWhUywF1', input={'q': 'the-favourite-game.com'}, name='_search', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 2409; Out: 104; Cache create: 0; Cache read: 0; Total: 2513)
Message(id='msg_01UHPMNbhzwfwqMwGqzs8Qk3', content=[TextBlock(text="Great news! The domain 'the-favourite-game.com' is available for purchase. The price is 1105 cents (USD), which is equivalent to $11.05.\n\nNow that we've confirmed its availability, let's proceed with the purchase request using the credit card payment method. Before we do that, we need to make sure that the contact information is set up, as it'

Excellent! The purchase request for 'the-favourite-game.com' has been successfully initiated using the credit card payment method. Here's what you need to know:

1. A checkout URL has been generated for you to complete the payment.
2. The payment link will expire on 2025-01-12T13:12:27.059Z (about 30 minutes from now).

To complete the purchase, please follow these steps:

1. Click on the following checkout URL to process your payment:
   https://checkout.stripe.com/c/pay/cs_live_a1Qbme07ZkdJqO5lyJzTTvAuCUtLqHHFaZXy3s67YbRk1DU347mQpkspWW#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl

2. Complete the payment process on the Stripe checkout page.

3. Once the payment is processed successfully, the domain 'the-favourite-game.com' will be registered to your account.

Please make sure to complete the payment before the expiration time to secure your domain. If you have any issues with the payment process or need any further assistance, please let me know.

<details>

- id: `msg_01HGMiMRxpTPDAaJMLC14SUC`
- content: `[{'text': "Excellent! The purchase request for 'the-favourite-game.com' has been successfully initiated using the credit card payment method. Here's what you need to know:\n\n1. A checkout URL has been generated for you to complete the payment.\n2. The payment link will expire on 2025-01-12T13:12:27.059Z (about 30 minutes from now).\n\nTo complete the purchase, please follow these steps:\n\n1. Click on the following checkout URL to process your payment:\n   https://checkout.stripe.com/c/pay/cs_live_a1Qbme07ZkdJqO5lyJzTTvAuCUtLqHHFaZXy3s67YbRk1DU347mQpkspWW#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl\n\n2. Complete the payment process on the Stripe checkout page.\n\n3. Once the payment is processed successfully, the domain 'the-favourite-game.com' will be registered to your account.\n\nPlease make sure to complete the payment before the expiration time to secure your domain. If you have any issues with the payment process or need any further assistance, please let me know.", 'type': 'text'}]`
- model: `claude-3-5-sonnet-20240620`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 3905, 'output_tokens': 498}`

</details>